In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd

In [2]:
'''def read_images():
    training_images = []
    i = 0
    DIR = "E:\Galaxy Zoo\Data\images_training_rev1\images_training_rev1"
    for img in os.listdir(DIR):
        im = cv2.imread(os.path.join(DIR,img))
        im = im[106:106*3,106:106*3,:]          #cropping 424x424->212x212
        im = cv2.resize(im,(106,106), interpolation=cv2.INTER_CUBIC)   #downsampling to half resolution 106x106
        training_images.append(im)
        i+=1
        if i==10000:
            break
        
    return training_images'''
    

'def read_images():\n    training_images = []\n    i = 0\n    DIR = "E:\\Galaxy Zoo\\Data\\images_training_rev1\\images_training_rev1"\n    for img in os.listdir(DIR):\n        im = cv2.imread(os.path.join(DIR,img))\n        im = im[106:106*3,106:106*3,:]          #cropping 424x424->212x212\n        im = cv2.resize(im,(106,106), interpolation=cv2.INTER_CUBIC)   #downsampling to half resolution 106x106\n        training_images.append(im)\n        i+=1\n        if i==10000:\n            break\n        \n    return training_images'

In [3]:
DIR = "E:\Galaxy Zoo\Data\images_training_rev1"

train_path = DIR + "\Train"
validate_path = DIR + "\Validate"

df = pd.read_csv("training_solutions_rev1/training_solutions_rev1.csv")

df.set_index('GalaxyID', inplace = True)

In [4]:
def img_processor(path):
    img = cv2.imread(path)
    img = img[106:106*3,106:106*3,:]
    img = cv2.resize(img,(106,106), interpolation = cv2.INTER_CUBIC)
    return img

def Batch_generator(DIR):
    for img in os.listdir(DIR):
        im = img_processor(os.path.join(DIR,img))
        ind = int(img[:-4])
        y_train = df.loc[ind].values
        X_train = np.array(im)
        X_train = X_train.reshape(1,106,106,3)
        y_train = y_train.reshape(1,37)
        yield(X_train,y_train)
        
def Validation_generator(DIR):
    for img in os.listdir(DIR):
        im = img_processor(os.path.join(DIR,img))
        ind = int(img[:-4])
        y_valid = df.loc[ind].values
        X_valid = np.array(im)
        X_valid = X_valid.reshape(1,106,106,3)
        y_valid = y_valid.reshape(1,37)
        yield(X_valid,y_valid)

In [5]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D,Lambda
from keras.optimizers import SGD, RMSprop, Adam

Using TensorFlow backend.


In [6]:
def Conv(model,layers,filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1))) 
        model.add(Conv2D((filters), kernel_size = (3,3), activation = 'relu'))
        
    model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
    
def FC_block(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    
def VGG16():
    model = Sequential()
    model.add(Lambda(lambda x : x, input_shape=(106,106,3)))

    Conv(model,2,64)
    Conv(model,2,128)
    Conv(model,3,256)
    Conv(model,3,512)
    Conv(model,3,512)
    
    model.add(Flatten())
    FC_block(model)
    FC_block(model)
    
    model.add(Dense(37))
    model.add(Activation("sigmoid"))

    return model

In [7]:

model = VGG16()

model.compile(optimizer = RMSprop(lr=1e-6), loss='mean_squared_error', metrics = ['accuracy'])


W0810 00:12:20.509856  8776 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 00:12:20.524849  8776 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 00:12:20.529799  8776 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 00:12:20.553733  8776 deprecation_wrapper.py:119] From c:\users\shinigami shrek\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3976: 

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 106, 106, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 108, 108, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 106, 106, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 108, 108, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 64)        0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 55, 55, 64)        0         
__________

In [9]:
batch_size = 64

steps_per_batch = int(len(os.listdir(train_path))/batch_size)
val_steps_per_batch = int(len(os.listdir(validate_path))/batch_size)

In [ ]:
model.fit_generator(Batch_generator(train_path),
                    samples_per_epoch=steps_per_batch,
                    epochs = 5,
                    validation_data=Validation_generator(validate_path),
                    nb_val_samples=val_steps_per_batch,
                    verbose = 1
                   )